## Python Imports

In [ ]:
!pip install accelerate
!pip install transformers
!pip install sentence-transformers
!pip install datasets
# !pip -q install google-generativeai==0.3.0
# !pip -q install google-ai-generativelanguage==0.4.0
import accelerate
import torch
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

import os
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset, DatasetDict
from accelerate import Accelerator
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Read File

In [ ]:
# # Load the annotation file
# annotation_file = '/content/subtask-3-annotations.txt'
# text_files_dir = '/content/'

# df = pd.read_csv(annotation_file, sep='\t', header=None, names=['File Name', 'Narrative 1', 'Narrative 2', 'Explanation'])
# # Function to get the content of a file
# def get_file_content(file_name):
#     file_path = os.path.join(text_files_dir, file_name)
#     if os.path.exists(file_path):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     else:
#         return "File not found"

# # Replace file names with actual file content
# df['Content'] = df['File Name'].apply(get_file_content)
# df['Prompt'] = np.where(df['Narrative 2'] != 'none',
#                         'Based on the narrative following narrative: ' + df['Narrative 2']+ '  find the summary of the: ' + df['Content'],
#                         'Based on the narrative following narrative: ' + df['Narrative 1']+ '  find the summary of the: ' + df['Content'])
# df

In [ ]:
# tedf.to_csv('testing.csv', index=False)

In [ ]:
# df.to_csv('tqsk3.csv', index=False)
df = pd.read_csv('/content/tqsk3.csv')

# Training, validation and testing data split

In [ ]:
tdf = df.iloc[:70] #70
vdf = df.iloc[70:77] #7
tedf = df.iloc[77:] #12

# Model and Tokenizing

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

checkpoint = "facebook/bart-large"
model = BartForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = BartTokenizer.from_pretrained(checkpoint)

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
from datasets import Dataset
import numpy as np

def tokenize_function(example):
    inputs = tokenizer(example['Prompt'], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(example['Explanation'], max_length=80, truncation=True, padding="max_length", return_tensors="pt")

    inputs['labels'] = targets['input_ids']
    return inputs

# df['Prompt'] = np.where(df['Narrative 2'] != 'none',
#                         'Based on the narrative following narrative: ' + df['Narrative 2']+ '  find the summary of the: ' + df['Content'],
#                         'Based on the narrative following narrative: ' + df['Narrative 1']+ '  find the summary of the: ' + df['Content'])

df_train = tdf[['Prompt', 'Explanation']].copy()
df_val = vdf[['Prompt', 'Explanation']].copy()

traindataset = Dataset.from_pandas(df_train)
train_dataset = traindataset.map(tokenize_function, batched=True)

validationdataset = Dataset.from_pandas(df_val)
validation_dataset = validationdataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

# Metrics

In [ ]:
! pip install rouge_score
! pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=631659aacceba9a5b0694b3f4b239c2b452227d337751c976ecb8b370b7f2a03
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import numpy as np

# Use evaluate instead of load_metric
from evaluate import load

# NLTK imports for BLEU
from nltk.translate.bleu_score import corpus_bleu
import nltk
nltk.download('punkt')  # Ensure 'punkt' is downloaded
from nltk.tokenize import word_tokenize

# Install 'rouge_score' outside the script using pip if needed, but not inside the script

# Load the metrics using evaluate
rouge_metric = load('rouge')

# BLEU isn't directly supported in evaluate, so we use nltk for BLEU
def compute_metrics(pred):
    pred_logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in pred_ids]

    # Remove ignored index (-100) from labels and replace with pad token id
    labels_ids = np.where(pred.label_ids != -100, pred.label_ids, tokenizer.pad_token_id)
    label_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels_ids]

    # Compute ROUGE score
    rouge_output = rouge_metric.compute(predictions=pred_str, references=label_str)

    # Prepare tokens for BLEU score
    pred_tokens = [nltk.word_tokenize(text) for text in pred_str]
    label_tokens = [[nltk.word_tokenize(text)] for text in label_str]

    # Compute BLEU score using NLTK
    bleu_output = corpus_bleu(label_tokens, pred_tokens)

    # Return ROUGE and BLEU scores
    return {
        'rouge1': rouge_output['rouge1'],   # No more .mid.fmeasure, just take the value
        'rouge2': rouge_output['rouge2'],
        'rougeLsum': rouge_output['rougeLsum'],
        # 'bleu': bleu_output,
    }


# Initialize WandB with disabled mode
import wandb
wandb.init(mode="disabled")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Training

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',           # Directory to save checkpoints
    num_train_epochs=4,               # Number of training epochs
    per_device_train_batch_size=2,
    warmup_steps=10,                  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=20,                 # Log every X updates
    save_strategy="epoch",            # Save the model at the end of each epoch
    eval_strategy="epoch",            # Evaluate at the end of each epoch
    load_best_model_at_end=True,      # Load the best model found during evaluation
    learning_rate=3e-5,               # Learning rate
    save_total_limit=5,
    metric_for_best_model="rougeLsum",   # Use ROUGE-L to determine the best model
    greater_is_better=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_length=80,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

generation_config.save_pretrained("/content/")


In [ ]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

model = AutoModelForSeq2SeqLM.from_pretrained("/content/")
tokenizer = AutoTokenizer.from_pretrained("/content/")
generation_config = GenerationConfig.from_pretrained("/content/")

In [ ]:
tedf

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def summarize_text(text):

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    summary_ids = model.generate(inputs['input_ids'], generation_config=generation_config)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

tedf['summary'] = tedf['Prompt'].apply(summarize_text)

In [ ]:
!pip install bert_score

In [ ]:
import pandas as pd
from evaluate import load

bertscore = load("bertscore")

predictions = tedf['summary'].tolist()
references = tedf['Explanation'].tolist()

results = bertscore.compute(predictions=predictions, references=references, lang="en",  model_type="distilbert-base-uncased")

In [ ]:
results

# Extra

In [ ]:
max_length = tedf['summary'].apply(len).max()

print(f'The maximum length of entries in "text_column" is: {max_length}')

In [ ]:
print(tedf['Explanation'].iloc[3])

In [ ]:
print(tedf['summary'].iloc[3])